### Needed imports 

In [ ]:
import bs4 as bs
from selenium import webdriver
from time import sleep
import json

### Set up

In [ ]:
URL = 'https://www.endomondo.com/'
LOGIN_ROUTE = 'login/'
list_of_workouts = []
USER_ID = ''
WORKOUT_LIMIT = 1000
EMAIL = 'email'
PASS = 'password'

### Log in  

In [ ]:
def log_in(EMAIL,PASS):
    sleep(2)
    driver.find_element_by_name('email').send_keys(EMAIL)
    sleep(3)
    driver.find_element_by_name('password').send_keys(PASS)
    sleep(1)
    buttons = driver.find_elements_by_xpath("//*[contains(text(), 'LOG IN')]")
    for btn in buttons:
        btn.click()
        sleep(5)

### Get user ID

In [ ]:
def get_user_id():
    HISTORY = ''
    html = driver.execute_script("return document.documentElement.outerHTML")
    sel_soup = bs.BeautifulSoup(html, 'html.parser')
    for link in sel_soup.findAll('a'):
        if '/history' in str(link.get('href')):
            HISTORY = link.get('href')[1:]
    USER_ID_TMP = '' 
    USER_ID_TMP = HISTORY.replace('users/', '').replace('/history', '')
    return USER_ID_TMP

### Load all workouts 

In [ ]:
def load_all_workouts(USER_ID,WORKOUT_LIMIT):
    list_of_workouts_tmp = []
    ALL_HISTORY = 'rest/v1/users/{}/workouts/history?limit={}'.format(USER_ID,str(WORKOUT_LIMIT))
    driver.get(URL+ALL_HISTORY)
    sleep(5)
    html_hist = driver.execute_script("return document.documentElement.outerHTML")
    sleep(5)
    sel_soup_hist = bs.BeautifulSoup(html_hist, "html.parser")
    table_hist = sel_soup_hist.find('pre').text
    
    table_hist_list = json.loads(table_hist).get('data')
    for workout in table_hist_list:
        list_of_workouts_tmp.append(workout.get('id'))
    return list_of_workouts_tmp
        

### Get all workouts

In [ ]:
def get_all_workouts(input_list):
    for w in input_list:
        driver.get(URL + '/rest/v1/users/{}/workouts/{}/export?format=GPX'.format(USER_ID,w))
        sleep(5)

### Run all

In [ ]:
driver = webdriver.Chrome()
driver.get(URL+LOGIN_ROUTE)

log_in(EMAIL,PASS)
USER_ID = get_user_id()

list_of_workouts = load_all_workouts(USER_ID,WORKOUT_LIMIT)
get_all_workouts(list_of_workouts)

